In [90]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [91]:
#Read in the csv
lending_data = pd.read_csv("LoanStats_2019Q1.csv")

/Users/kenmurakami/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,123,124,125,128,129,130,133,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [92]:
lending_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000.0,20000.0,20000.0,60 months,17.19%,499.10,C,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,12000.0,12000.0,12000.0,36 months,16.40%,424.26,C,C4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3000.0,3000.0,3000.0,36 months,14.74%,103.62,C,C2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,35000.0,35000.0,35000.0,36 months,15.57%,1223.08,C,C3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,5000.0,5000.0,5000.0,36 months,15.57%,174.73,C,C3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
lending_df = lending_data[["loan_status","grade","sub_grade","annual_inc","dti","addr_state","int_rate"]]
status = lending_df.groupby("loan_status").count()
status

,grade,sub_grade,annual_inc,dti,addr_state,int_rate
loan_status,,,,,,
Charged Off,176,176,176,176,176,176
Current,109176,109176,109176,108936,109176,109176
Fully Paid,4730,4730,4730,4716,4730,4730
In Grace Period,538,538,538,537,538,538
Late (16-30 days),260,260,260,259,260,260
Late (31-120 days),795,795,795,794,795,795


In [162]:
#Create the upsample and downsample
small_sample = lending_df[(lending_df["loan_status"] == "Charged Off") | (lending_df["loan_status"] 
                                                                        == "Late (31-120 days)")]
large_sample = lending_df[(lending_df["loan_status"] == "Fully Paid")]


In [165]:
small_sample.head()
large_sample.head()

,loan_status,grade,sub_grade,annual_inc,dti,addr_state,int_rate
64,Fully Paid,C,C3,65000.0,15.18,CO,15.57%
87,Fully Paid,C,C5,82000.0,15.70,MO,17.19%
91,Fully Paid,C,C1,90000.0,7.92,NY,13.90%
105,Fully Paid,D,D5,17808.0,30.12,OH,28.80%
133,Fully Paid,B,B4,38000.0,38.66,GA,12.40%


In [172]:
n_class0 = len(small_sample)
n_class1 = len(large_sample)

under = large_sample.sample(n_class0)
sample_df = pd.concat([under, small_sample], axis=0)

len(sample_df)


1942

In [173]:
X = sample_df[["grade", "sub_grade","annual_inc", "dti", "int_rate"]]
y = sample_df["loan_status"].values.reshape(-1,1)
print(X.shape, y.shape)

(1942, 5) (1942, 1)


In [179]:
#making loan_status binary

data_binary_encoded = pd.get_dummies(sample_df, columns=["loan_status"])
pd.data_binary_encoded


,grade,sub_grade,annual_inc,dti,addr_state,int_rate,loan_status_Charged Off,loan_status_Fully Paid,loan_status_Late (31-120 days)
1269,D,D1,45000.0,26.77,NJ,17.97%,0,1,0
77523,C,C1,65000.0,11.63,NY,13.56%,0,1,0
17572,C,C5,85500.0,18.02,NY,17.19%,0,1,0
107005,D,D3,48200.0,56.03,MO,19.92%,0,1,0
104501,E,E5,82000.0,15.31,NY,27.27%,0,1,0
2448,C,C2,70000.0,15.19,FL,14.74%,0,1,0
17109,C,C4,127000.0,11.20,NY,16.40%,0,1,0
58036,D,D4,86000.0,31.97,CA,20.89%,0,1,0
104770,C,C1,75000.0,18.93,OH,13.56%,0,1,0
99772,A,A5,72000.0,22.33,MN,8.81%,0,1,0
